# Availability 

In this notebook, the proteins and genes extracted from the Proteomics.ipynb and RNA-seq.ipynb are vetted for aviaibaility. 

This was added due to inconsistent results in the Final notebook and was proven to be essential to the process. Genes that are not available for at least 72 out of 80 samples (90%) are discarded in the final classification step. 

In [1]:
!ls
#######################################################################

Final Results.ipynb MSI Visual.ipynb    __pycache__         raw_data
Gender Visual.ipynb Proteomics.ipynb    availability.ipynb  subchallenge_2.csv
Import.ipynb        RNA-Seq.ipynb       preprocess.py


In [2]:
import sys
import os
from IPython.utils import io
from io import StringIO

import re
import csv

import pandas as pd
import numpy as np
import scipy 
from scipy.cluster import hierarchy as hc # for dendograms 

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# %matplotlib notebook

from sklearn.metrics import f1_score # f1_score(y_true, y_pred)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectFromModel

## Import & show data 

The initial data proprocess was done in the Import.ipynb notebook. The following three steps are performed in all other jupyter notebooks: run the preprocess.py code that was generated in Import.ipynb, run the %whos magic command to see what dataframes have been imported, and then format the captured %who results with pandas to view results. 

In [3]:
%run preprocess.py
# made in Import.ipynb

In [4]:
with io.capture_output() as captured:
    %whos DataFrame 

s = re.findall(r'\S+|\n', StringIO(captured.stdout).getvalue())

In [5]:
table = []
row = []

for item in s:
    if (item == '\n'):
        if (row):
            table.append(row)
        row = []
    elif re.match('^(?!-)', item) is None:
        pass        
    else: 
        row.append(item)

for item in table: 
    if (item[0] == 'Variable'):
        pass
    else:
        item[2:] = [' , '.join(item[2:])]

df = pd.DataFrame(data=table[1:], columns=table[0])
pd.set_option('display.max_colwidth', -1)
df

,Variable,Type,Data/Info
0,df_test_cli,DataFrame,"sample , gender , <...>g_80 , Female , MSI-High"
1,df_test_pro,DataFrame,"A1BG , <...>n[80 , rows , x , 4118 , columns]"
2,df_test_rna,DataFrame,"A1BG , A1B<...>[80 , rows , x , 17447 , columns]"
3,df_train_cli,DataFrame,"sample , gender , <...>g_80 , Female , MSI-Low/MSS"
4,df_train_mislabel,DataFrame,"sample , mismatch<...>79 , Training_80 , 1"
5,df_train_pro,DataFrame,"A1BG , <...>n[80 , rows , x , 4118 , columns]"
6,df_train_rna,DataFrame,"A1BG , A1<...>[80 , rows , x , 17447 , columns]"
7,test_cli,DataFrame,"gender , msi\n<...>\nTesting_80 , 1 , 1"
8,test_pro,DataFrame,"A1BG , <...>n[80 , rows , x , 4118 , columns]"
9,test_rna,DataFrame,"A1BG , A1B<...>[80 , rows , x , 17447 , columns]"


In [7]:
# train_pro_combined
# train_rna_combined
# train_combined.head()

#### These are extracted from Proteomics and RNA-Seq notebooks, many of the non-available genes were discarded in the porcess and these are only left now; meaning the current output does not reflect the process and work that was done to dicard those genes.

In [10]:
train_combined[
    
    [ # msi proteomics
    'TAP1', 'APOL2', 'LCP1', 'PTPN6', 'CASK', 'UBE2L6', 
    'ICAM1', 'ITGB2', 'HP', 'HK3', 'RFC2', 'WARS',
    'SDF2L1', 'CKB', 'LAP3', 'IFI30', 'PTPRC', 'HSDL2', 
    'IFI35', 'TYMP', 'CSRP2', 'TAPBP', 'ERMP1', 'ANP32E', 
    'ROCK2', 'ADPGK', 'HSPA4L', 'CNDP2', 'RFTN1', 'GBP1', 
    'GBP2', 'FMNL1', 'DMBT1', 'ARL3',
    'NCF2', 'YARS2', 'RPL3', 'SPTLC2', 'ENO1', 'SNX12', 
                
    # mis rna-seq 
    'EPDR1', 'APOL3', 'POU5F1B', 'CFTR', 'CIITA', 'RAB32', 
    'MAX', 'TRIM72', 'GBP4', 'HPSE', 'WARS', 'GRM8', 'SLC51B',
    'PRSS23', 'FABP6', 'GABRP', 'LAP3', 'LY6G6D', 'SLC19A3', 
    'DLGAP1-AS5', 'GBP1', 'RAMP1', 'AREG', 'GSPT2', 'TNFAIP2',
    'EREG', 'TNNC2', 'ANKRD27', 'PLCL2', 'TFCP2L1', 'LAG3', 
    'BEX2', 'DEFB1', 'FABP1', 'IRF1', 'CCL4', 'SLC25A48', 

    # gender proteomics
    'MYO9B', 'EMILIN1', 'MRPL16', 'EIF4G2', 'TNXB', 'ARFIP1', 
    'GAR1', 'SLC35A2', 'FBLN1', 'FLNC', 'S100A14', 'COX7C', 
    'FBN1', 'C4A','COL4A2','RCC2','ZNF706', 'NAP1L1', 'AGRN', 
    'GADD45GIP1', 'TJP2', 'ALDH1A1', 'PGM2', 'LTBP1', 'NNMT', 
    'SERPINB1', 'U2AF1L5', 'COG3', 'ASPN', 'METTL1', 'DDT', 

    # gender rna-seq 
    'RPS4Y1', 'ZFX', 'DDX3Y', 'UTY', 'SRSF6', 'GYG2P1', 'DDB2', 
    'ZRSR2', 'EMX1', 'EHBP1', 'THG1L', 'KCNJ2', 'ZNF606',
    'EVPL', 'LUC7L2', 'SRGAP2B', 'NAT8', 'ZNF862', 'ODAM', 
    'RNASEH2A', 'EIF1AY', 'FOXD2', 'PMS2P3', 'MND1', 'FAM3D', 
    'DDX3X', 'NINJ2', 'NLRX1', 'COCH', 'IRF5', 'FHDC1', 
    'FBXW9', 'RPL21P28', 'TLX1', 'HEBP2', 'TRIM29', 'ZNF273', 
    'CRABP2', 'ABCA6', 'MMP17', 'PCNA', 'S100A16', 'UBA6', 'RGS4', 
    ]
    
            ].isnull().sum(axis = 0)

TAP1        0
TAP1        0
APOL2       0
APOL2       0
LCP1        0
LCP1        0
PTPN6       0
PTPN6       0
CASK        0
CASK        0
UBE2L6      0
UBE2L6      0
ICAM1       0
ICAM1       0
ITGB2       0
ITGB2       0
HP          0
HP          0
HK3         0
HK3         0
RFC2        0
RFC2        0
WARS        0
WARS        0
SDF2L1      0
SDF2L1      0
CKB         0
CKB         0
LAP3        0
LAP3        0
IFI30       0
IFI30       0
PTPRC       0
PTPRC       0
HSDL2       0
HSDL2       0
IFI35       0
IFI35       0
TYMP        0
TYMP        0
CSRP2       0
CSRP2       0
TAPBP       0
TAPBP       0
ERMP1       0
ERMP1       0
ANP32E      0
ANP32E      0
ROCK2       0
ROCK2       0
           ..
SRSF6       0
GYG2P1      0
DDB2        0
ZRSR2       0
EMX1        0
EHBP1       0
THG1L       0
KCNJ2       0
ZNF606      0
EVPL        0
LUC7L2      0
LUC7L2      0
SRGAP2B     0
NAT8        0
ZNF862      0
ODAM        0
RNASEH2A    0
RNASEH2A    0
EIF1AY      0
EIF1AY      0
FOXD2 